In [8]:
from IPython.core.display import HTML
HTML("""
<style>
h1, .h1 {
    font-size: 33px;
    font-family: "Trebuchet MS";
    font-size: 1.5em !important;
    color: #2a7bbd;
}}
</style>
""")

# Prepare

## imports

In [9]:
%load_ext autoreload
%autoreload 2

import warnings

import numpy as np
import pandas as pd
import os
import sys

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

pd.set_option('display.float_format', lambda x: '%.5f' % x)

np.random.seed(0)

In [10]:
module_path = os.path.abspath(os.path.join('../src'))
print("Adding modules", module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

Adding modules /Users/christopherlomeli/Source/courses/datascience/nfl_capstone/src


In [11]:
from src.features.wrangling.database_loader import DatabaseLoader
from src.features.wrangling.get_metrics import GetMetrics, get_versioned_name, conform_column_names

# setup

In [12]:
RAW_DATA_PATH = '../data/raw'
INTERIM_DATA_PATH='../data/interim'

# inputs
INPUT_DATA=os.path.join(RAW_DATA_PATH,"nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv")

#outputs
OUTPUT_DATA = os.path.join(INTERIM_DATA_PATH,"nflplaybyplay2009to2016_reviewed_01.parquet")
BACKUP_METRICS_FILE = os.path.join(INTERIM_DATA_PATH,"metrics_backup.parquet")
DIMENSIONS_DATA = os.path.join(INTERIM_DATA_PATH,"dimensions.parquet")

# database
METRICS_TABLE_NAME="nfl_metrics"
CATEGORIES_TABLE_NAME="nfl_categories"

# verify
print("INPUT_DATA", INPUT_DATA)
print("OUTPUT_DATA", OUTPUT_DATA )
print("BACKUP_METRICS_FILE", BACKUP_METRICS_FILE )
print("", DIMENSIONS_DATA)
print("DIMENSIONS_DATA", METRICS_TABLE_NAME)
print("CATEGORIES_TABLE_NAME", CATEGORIES_TABLE_NAME)


INPUT_DATA ../data/raw/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv
OUTPUT_DATA ../data/interim/nflplaybyplay2009to2016_reviewed_01.parquet
BACKUP_METRICS_FILE ../data/interim/metrics_backup.parquet
 ../data/interim/dimensions.parquet
DIMENSIONS_DATA nfl_metrics
CATEGORIES_TABLE_NAME nfl_categories


In [13]:
db = DatabaseLoader(connection_string_env_url="DB_CONNECTION_URL")

In [14]:
# rdf = db.read_table("nfl_dim")
# rdf
# rdf.to_parquet(DIMENSIONS_DATA, engine='fastparquet',  compression='snappy')

In [15]:
# Creating the Series
sr = pd.Series(['Mike', 'Alessa', 'Nick', 'Kim', 'Britney'])

# Creating the index
idx = ['Name 1', 'Name 2', 'Name 3', 'Name 4', 'Name 5']

# set the index
sr.index = idx

# Print the series
sr




Name 1       Mike
Name 2     Alessa
Name 3       Nick
Name 4        Kim
Name 5    Britney
dtype: object

In [16]:
# extract groups having any capital letter
# followed by 'i' and any other character
result = sr.str.extract(pat = '([A-Z]i.)')

result

,0
Name 1,Mik
Name 2,NaN
Name 3,Nic
Name 4,Kim
Name 5,NaN


In [17]:
# Import needed package
import pycodestyle

# Create a StyleGuide instance
style_checker = pycodestyle.StyleGuide()

In [18]:
import boto3

# key = "nfl_capstone"
AWS_S3_BUCKET = "cjl-project-data"
#
# s3_client = boto3.client()

import boto3
s3 = boto3.resource('s3')
my_bucket = s3.Bucket(AWS_S3_BUCKET)

for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object)

s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/external/.gitkeep')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/processed/.gitkeep')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/raw/dimensions.csv')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/raw/games.csv')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/raw/nfl_stadiums.csv')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/raw/nfl_teams.csv')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/raw/nfl_teams_scraped.csv')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/raw/nflplaybyplay2009to2016/NFL Play by Play 2009-2016 (v3).csv')
s3.ObjectSummary(bucket_name='cjl-project-data', key='nfl_capstone/data/raw/nflplaybyplay2009to2016/NFL Play by Play 2009-2017 (v4).csv')
s

In [19]:
import nfl_data_py as nfl
pbp = nfl.import_pbp_data([2009],  downcast=True, cache=False, alt_path=None)
type(pbp)

2009 done.
Downcasting floats.


pandas.core.frame.DataFrame

In [24]:
pbp = pbp.head(10)


In [25]:
db.load_table(pbp, 'nflverse')

In [26]:
pbp.dtypes


play_id                float32
game_id                 object
old_game_id             object
home_team               object
away_team               object
                        ...   
xyac_median_yardage    float32
xyac_success           float32
xyac_fd                float32
xpass                  float32
pass_oe                float32
Length: 372, dtype: object

In [30]:
pbp['defensive_two_point_attempt'].value_counts(dropna=False)

0.00000    9
NaN        1
Name: defensive_two_point_attempt, dtype: int64